# Modelos de Cáculo

## CFG

In [ ]:
import nltk
from nltk import CFG

def probar_gramatica():
    # Definir la gramática libre de contexto
    gramatica = CFG.fromstring("""
    S -> A | B
    A -> "0" A "2" | "1" B "2" | ""
    B -> "1" B "2" | ""
    """)

    # Crear el parser para la gramática
    parser = nltk.ChartParser(gramatica)

    # Definir una oración de ejemplo
    oracion = "0122"
    palabras = oracion.split()  # Asegúrate de que las palabras estén separadas

    # Intentar analizar la oración
    print(f"Probando la oración: {oracion}")
    for arbol in parser.parse(palabras):
        print(arbol)



probar_gramatica()


In [ ]:
import nltk
from nltk import CFG
import itertools

def generar_cadenas(gramatica, simbolo_inicial='S', max_longitud=5):
    """
    Genera todas las strings posibles de la gramática hasta una longitud máxima.
    
    :param gramatica: La gramática en formato CFG.
    :param simbolo_inicial: El símbolo inicial (por defecto 'S').
    :param max_longitud: La longitud máxima de las strings generadas.
    :return: Una lista con todas las strings posibles.
    """
    producciones = gramatica.productions()
    print(producciones)
    def es_terminal(simbolo):
        """Verifica si un símbolo es terminal."""
        return simbolo in gramatica._lexical_index
    
    def generar_desde(simbolo, longitud_actual=0):
        """Genera recursivamente las cadenas a partir de un símbolo no terminal."""
        if es_terminal(simbolo):
            return [[simbolo]]  # Devuelve la cadena terminal como lista

        if longitud_actual >= max_longitud:
            return []  # No generar más si se ha alcanzado la longitud máxima

        cadenas = []
        for produccion in producciones:
            if produccion.lhs() == simbolo:
                partes = produccion.rhs()
                subcadenas = [generar_desde(s, longitud_actual + 1) for s in partes]
                for combinacion in itertools.product(*subcadenas):
                    cadenas.append([item for sublista in combinacion for item in sublista])
        return cadenas
    
    # Generar todas las cadenas desde el símbolo inicial
    todas_cadenas = generar_desde(gramatica.start())
    return [' '.join(cadena) for cadena in todas_cadenas]

# Definir la gramática libre de contexto
gramatica = CFG.fromstring("""
S -> A | B
A -> '0' A '2' | '1' B '2' | ""
B -> '1' B '2' | ""
""")

# Generar cadenas hasta longitud 6
cadenas_generadas = generar_cadenas(gramatica, max_longitud=39)

# Imprimir las cadenas generadas
for cadena in cadenas_generadas:
    ceros = cadena.count('0')
    unos = cadena.count('1')
    doses = cadena.count('2')
    print(f'{ceros} + {unos} =? {doses}', ceros + unos == doses)
    print(f"Cadena generada: '{cadena}'")


## Algoritmos de Markov

In [3]:
class algoritmo_de_Markov:
    def __init__(self, reglas, alfabeto =None):
        if alfabeto is None:
            self.alfabeto = [elem for regla in reglas for elem in regla[:2]]
        else:
            self.alfabeto = set(alfabeto)
        self.reglas = reglas
        self._check_reglas()
        pass

    def _check_reglas(self):
        existe_terminal = False
        num_regla = 1
        for patron, remplazo, ind_terminal in self.reglas:
            for i in patron:
                if i not in self.alfabeto:
                    raise ValueError(f"Regla {num_regla}: El símbolo '{i}' no pertenece al alfabeto")
            for j in remplazo:
                if j not in self.alfabeto:
                    raise ValueError(f"Regla {num_regla}: El símbolo '{j}' no pertenece al alfabeto")
            
            if ind_terminal:
                existe_terminal = True
            
            num_regla += 1

        if not existe_terminal:
            raise ValueError("No hay reglas terminales")
        pass

    def _check_string(self, string):
        for i in string:
            if i not in self.alfabeto:
                raise ValueError(f"String: El símbolo '{i}' no pertenece al alfabeto")
        
    def exec_string(self, string, verbose=False):
        self._check_string(string)
        while True:
            aplicada = False  # Bandera para verificar si alguna regla fue aplicada
            for num_regla, (patron, remplazo, ind_terminal) in enumerate(self.reglas, start=1):
                pos = string.find(patron)
                if pos != -1:
                    string = string[:pos] + remplazo + string[pos + len(patron):]
                    aplicada = True
                    if verbose:
                        print(f"Estado string: {string}")
                        print(f'Se aplica la regla {num_regla}: "{patron}" -> "{remplazo}"')
                    if ind_terminal:
                        if verbose:
                            print("Regla terminal aplicada, deteniendo ejecución.")
                        return string  # Si es terminal, termina aquí
                    break  # Reinicia desde la primera regla
            if not aplicada:
                # Si no se aplicó ninguna regla en este ciclo, termina la ejecución
                if verbose:
                    print("No se ha podido aplicar ninguna regla más")
                return string

In [10]:
#Ejemplo 2
alfabeto = ['a', 'b', 'X', 'Y']
R = [
    ("ab", "X", False),
    ("ba", "Y", True)
]

modelo = algoritmo_de_Markov(alfabeto, R)

print(modelo.alfabeto)
s= 'ababba'
print(s)
print(modelo.exec_string(s, verbose=True))


{'b', 'a', 'Y', 'X'}
ababba
Estado string: Xabba
Se aplica la regla 1: "ab" -> "X"
Estado string: XXba
Se aplica la regla 1: "ab" -> "X"
Estado string: XXY
Se aplica la regla 2: "ba" -> "Y"
Regla terminal aplicada, deteniendo ejecución.
XXY


In [ ]:
#Ejemplo 3

alfabeto = ['a', 'b', '', '0', '1']
R = [
    ('ba', '0', False),
    ('a0', '0', False),
    ('0a', '0', False),
    ('b0', '0', False),
    ('0b', '0', False),
    ('ab', '', False),
    ('a', '0', False),
    ('b', '0', False),
    ('0', '0', True),
    ('', '1', True)
]

pruebas = ['aaabbb', 'aabbb', 'aaa']

modelo = algoritmo_de_Markov(R, alfabeto)

for p in pruebas:
    if modelo.exec_string(p) == '1':
        print(f'La cadena {p} es valida')
    else:
        print(f'La cadena {p} no es valida')

In [7]:
# Ejemplo 4
import string

alfabeto = list(string.ascii_lowercase)  # Genera ['a', 'b', ..., 'z']
alfabeto.extend(['ñ',' ', ',', '.', '?'])  # Agrega xtras

print(alfabeto)
R = [
    ('esdto', 'esto', False),
    ('qeu', 'que', False),
    ('hoal', 'hola', False),
    ('cmo', 'como', False),
    ('algo', '', True)
]


modelo = algoritmo_de_Markov(reglas=R, alfabeto=alfabeto)

string = 'hoal, cmo estas? esdto qeu te envio tiene errores.'
modelo.exec_string(string, verbose=True)


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ñ', ' ', ',', '.', '?']
Estado string: hoal, cmo estas? esto qeu te envio tiene errores.
Se aplica la regla 1: "esdto" -> "esto"
Estado string: hoal, cmo estas? esto que te envio tiene errores.
Se aplica la regla 2: "qeu" -> "que"
Estado string: hola, cmo estas? esto que te envio tiene errores.
Se aplica la regla 3: "hoal" -> "hola"
Estado string: hola, como estas? esto que te envio tiene errores.
Se aplica la regla 4: "cmo" -> "como"
No se ha podido aplicar ninguna regla más


'hola, como estas? esto que te envio tiene errores.'

In [11]:
# Ejemplo 5
import string
# Definir el conjunto de reglas de conversión Morse a letras
# Cada regla tiene el formato (patron, remplazo, ind_terminal)
reglas_morse = [
    (".-", "A", False), ("-...", "B", False), ("-.-.", "C", False), ("-..", "D", False),
    (".", "E", False), ("..-.", "F", False), ("--.", "G", False), ("....", "H", False),
    ("..", "I", False), (".---", "J", False), ("-.-", "K", False), (".-..", "L", False),
    ("--", "M", False), ("-.", "N", False), ("---", "O", False), (".--.", "P", False),
    ("--.-", "Q", False), (".-.", "R", False), ("...", "S", False), ("-", "T", False),
    ("..-", "U", False), ("...-", "V", False), (".--", "W", False), ("-..-", "X", False),
    ("-.--", "Y", False), ("--..", "Z", False),
    # Números
    ("-----", "0", False), (".----", "1", False), ("..---", "2", False), ("...--", "3", False),
    ("....-", "4", False), (".....", "5", False), ("-....", "6", False), ("--...", "7", False),
    ("---..", "8", False), ("----.", "9", False),
    # Espacios entre letras y palabras en Morse
    ("/", " ", False)  # Define "/" para separar palabras en morse con un espacio en el mensaje
]

# Definir el alfabeto que incluye '.' y '-' para representar morse
alfabeto_morse = ['.', '-', ' ', '/']
alfabeto_ascii = list(string.ascii_uppercase)
alfabeto_numeros = list(string.digits)

# Crear una instancia del algoritmo de Markov para decodificar Morse
decodificador_morse = algoritmo_de_Markov(reglas_morse)
print(decodificador_morse.alfabeto)
# Decodificar un mensaje de ejemplo
mensaje_morse = ".- / -... / -.-."
resultado = decodificador_morse.exec_string(mensaje_morse, verbose=True)
print(f"Mensaje decodificado: {resultado}")


ValueError: No hay reglas terminales

In [ ]:
class AlgoritmoDeMarkov:
    def __init__(self, alfabeto, reglas):
        self.alfabeto = set(alfabeto)
        self.reglas = reglas
        self._check_reglas()

    def _check_reglas(self):
        existe_terminal = False
        for patron, remplazo, ind_terminal in self.reglas:
            for i in patron:
                if i not in self.alfabeto:
                    raise ValueError(f"El símbolo '{i}' no pertenece al alfabeto")
            for j in remplazo:
                if j not in self.alfabeto and j != "":
                    raise ValueError(f"El símbolo '{j}' no pertenece al alfabeto")
            if ind_terminal:
                existe_terminal = True
        if not existe_terminal:
            raise ValueError("No hay reglas terminales")

    def exec_string(self, input_string, max_longitud=10):
        while len(input_string) < max_longitud:
            aplicada = False
            for patron, remplazo, ind_terminal in self.reglas:
                pos = input_string.find(patron)
                if pos != -1:
                    input_string = input_string[:pos] + remplazo + input_string[pos + len(patron):]
                    aplicada = True
                    if ind_terminal:
                        return input_string
                    break
            if not aplicada:
                break
        return input_string

    def generar_cadenas_markov(self, input_string="S", max_iteraciones=10, longitud_max=10):
        palabras = []
        actual_string = input_string
        for _ in range(max_iteraciones):
            nuevo_string = self.exec_string(actual_string, max_longitud=longitud_max)
            if nuevo_string == actual_string:
                break
            palabras.append(nuevo_string)
            actual_string = nuevo_string
        return palabras

# Definir alfabeto y reglas de Markov
alfabeto = "012SX"
reglas = [
    ("S", "X", False),
    ("X", "0X2", False),
    ("X", "1X2", False),
    ("X", "", True)
]

# Crear una instancia del algoritmo de Markov
markov = AlgoritmoDeMarkov(alfabeto, reglas)

# Generar cadenas con el algoritmo de Markov
cadenas_markov = markov.generar_cadenas_markov(input_string="S", max_iteraciones=10, longitud_max=10)

# Mostrar las cadenas generadas y realizar el análisis cuantitativo
for cadena in cadenas_markov:
    ceros = cadena.count('0')
    unos = cadena.count('1')
    doses = cadena.count('2')
    print(f'{ceros} + {unos} =? {doses}', ceros + unos == doses)
    print(f"Cadena generada: '{cadena}'")


In [ ]:
# Definir alfabeto y reglas
alfabeto = "abcX"
reglas = [
    ("a", "bX", False),  # Reemplaza 'a' con 'bX'
    ("X", "cc", True),   # Reemplaza 'X' con 'cc' (regla terminal)
    ("b", "ab", False)   # Reemplaza 'b' con 'ab'
]

# Crear una instancia del algoritmo de Markov
markov = algoritmo_de_Markov(alfabeto, reglas)

# Generar palabras con longitud máxima de 10 caracteres
input_string = "aabb"
palabras_generadas = markov.generar_palabras(input_string, max_iteraciones=10, longitud_max=10, verbose=True)

# Mostrar las palabras generadas
print("\nPalabras generadas:")
for palabra in palabras_generadas:
    print(palabra)